In [1]:
from stim_backend import NoisyStimBackend

Here's a new backend. Note that this has three error probabilities, for single-qubit, two-qubit and measurement gates. Errors are depolarizing for the former two, and bit flip for the latter.

In [60]:
p = 1e-2
n = 8
backend = NoisyStimBackend(
    num_qubits=n,
    basis_gates = ["id", "h", "x", "y", "z", "s", "cx"],
    p1=0,
    p2=0,
    pm=p
)

Let's try it out on the following circuit.

In [61]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.x(0)
qc.cx(0,1)
qc.measure_all()

This should yield `11` with certainty. So the probability that it doesn't is the probabilty of error. What do we expect for this?

* p1=p, p2=0, pm=0: p/2 (since only half of single-qubit Paulis are bit flips) = 0.005 (for p=1%)
* p1=0, p2=p, pm=0: 3p/4 (since 3/4s of two-qubit Paulis are bit flips) = 0.075 (for p=1%)
* p1=0, p2=0, pm=p: 2*p (since there are two ways it can happen) = 0.02 (for p=1%)

In [62]:
shots = 1e5
probs = {string:count/shots for string, count in backend.run(qc,shots=shots).result().get_counts().items()}
print('Total error')
print(round(1-probs['11'],4))

Total error
0.0201


Try it out. It seems to work!

Now let's see how we do with more qubits.

In [63]:
n = 127
backend = NoisyStimBackend(
    num_qubits=n,
    basis_gates = ["id", "h", "x", "y", "z", "s", "cx"],
    p1=p,
    p2=p,
    pm=p
)

qc = QuantumCircuit(n)
qc.h(range(n))
qc.measure_all()

probs = {string:count/shots for string, count in backend.run(qc,shots=shots).result().get_counts().items()}

This only takes a few seconds!

Unfortunately it doesn't seem to work with MirrorRB. Maybe because this backend is made to use `backend.run()` whereas Qiskit uses the sampler.